# ML Final Report
### Ben Richardson and Kendra Noneman

#### Link for documentation: https://huggingface.co/docs/transformers/model_doc/clip

In [1]:
from PIL import Image
import requests
import torch
import numpy as np

from transformers import CLIPProcessor, CLIPModel

### Acoustic Model:

##### Hubert: 
"Forces the model to learn a combined acoustic and language model over the continuous inputs":
https://huggingface.co/docs/transformers/v4.24.0/en/model_doc/hubert#overview



### Multimodal Model:

##### CLIPModel:
Model configuration class with all the parameters of the model. Initializing with a config file does not load the weights associated with the model, only the configuration.
 
##### from_pretrained:
Changes the weights (including mapping hidden states to vocabulary, pruning and initializing weights, tying the weights between the input embeddings and the output embeddings, etc...)
https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/model#transformers.PreTrainedModel.from_pretrained

Options for CLIP pre-trained openai models (4 of them):
https://huggingface.co/openai

##### CLIPProcessor:
Constructs a CLIP processor which wraps a CLIP feature extractor and a CLIP tokenizer into a single processor

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image.show()

In [4]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)
print(inputs["input_ids"])
print(inputs["input_ids"].shape)


tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]])
torch.Size([2, 7])


In [5]:
# Find the n words with largest auditory cue, and then set the attention mask to 1 only for those words

inputs["attention_mask"] = torch.Tensor([np.zeros(inputs["input_ids"].shape[1]).tolist(),np.zeros(inputs["input_ids"].shape[1]).tolist()])

In [6]:
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [7]:
print(probs)

tensor([[0.5949, 0.4051]], grad_fn=<SoftmaxBackward0>)
